## 維度災難
#### 欄位太多，導致:  1.一般演算法無法負荷   2.欄位資料相互關聯，資料需求量呈指數成長
## 單純貝氏 (分類)
- <font color=blue size=2>因為採"獨立事件"的單純假設，因此欄位資料相互變為無關聯，避免了維度災難</font>
- <font color=blue size=2>雖然計算結果非現實機率，但因是比大小問題，因此並不會影響結果太多</font>
- <font color=blue size=2>應用: 垃圾信檢測</font>

#### <font color=red size=3>步驟一: 讀入資料集</font>

In [2]:
import pandas as pd
train_df = pd.read_csv("poem_train.csv", encoding="utf-8")
test_df = pd.read_csv("poem_test.csv", encoding="utf-8")
train_df

,作者,詩名,內容
0,李白,菩薩蠻·平林漠漠煙如織,平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1,李白,把酒問月,青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2,李白,春思,燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3,李白,春夜洛城聞笛,誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4,李白,古風 其十九,西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
...,...,...,...
2726,白居易,彆元九後詠所懷,零落桐葉雨，蕭條槿花風。悠悠早秋意，生此幽閒中。況與故人彆，中懷正無悰。勿雲不相送，心到青門...
2727,白居易,早秋曲江感懷,離離暑雲散，嫋嫋涼風起。池上秋又來，荷花半成子。朱顏易銷歇，白日無窮已。人壽不如山，年光急於...
2728,白居易,東墟晚歇 時退居渭村。,涼風冷露蕭索天，黃蒿紫菊荒涼田。繞塚秋花少顏色，細蟲小蝶飛翻翻。中有騰騰獨行者，手拄漁竿不騎...
2729,白居易,南秦雪,往歲曾為西邑吏，慣從駱口到南秦。\r\n三時雲冷多飛雪，二月山寒少有春。\r\n我思舊事猶惆...


#### <font color=red size=3>步驟二: 將答案標籤轉換為數字字典，再準備一份反向字典</font>
- 因傳入必為數字，因此將作者(answer)做轉換
- 之後檢驗答案也會用到

In [3]:
# 準備一個轉換的字典
trans = {
    "李白":0,
    "杜甫":1,
    "白居易":2
}

# 再準備一個反向的字典
# 字典.items(): 把字典組合成tuple -> [('李白', 0), ('杜甫', 1), ('白居易', 2)]
reverse_trans = {v:k for k, v in trans.items()}
reverse_trans

{0: '李白', 1: '杜甫', 2: '白居易'}

In [6]:
# 將訓練集和測試集的answer(作者)都取代為數字
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)

#### <font color=red size=3>步驟三: 定義分詞函式，並將訓練和測試data做分詞</font>

In [8]:
# 先試分詞結果，再評估是否需要載入大辭典
# 是否刪除，。\n\r 等符號? 可觀察之後的函式庫會不會幫我們去掉
import jieba

def poemcut(s):
    s = " ".join(jieba.cut(s))  #把字串用空白做切割
    s = s.replace("\r", "").replace("\n","") # 把 \r 變空字串  \n 變空字串
    return s

# poemcut不加小括號: 對每一個都做一樣的事
train = train_df["內容"].apply(poemcut)  # apply=> 對每一格做一次那個流程
test = test_df["內容"].apply(poemcut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\torot\AppData\Local\Temp\jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built successfully.


#### <font color=red size=3>步驟四: 使用CountVectorizer做詞袋矩陣</font>
- fit: 只"確定欄位"，即決定欄位有什麼 -> 把不重複的詞抓出 
- transfrom: 真正做轉換 -> 把每個詞的次數數出來
- 注意: 測試資料不做fit -> 直接用訓練資料產生的欄位(詞)(在vec.vocabulary_)，若遇到訓練資料沒出現過的詞就直接忽略!!!!!

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(train)

# 測試資料不做fit
x_test = vec.transform(test)

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [13]:
vec.vocabulary_
# {欄位(詞): 對應的號碼}  

{'平林': 16053,
 '漠漠': 29006,
 '煙如織': 30177,
 '寒山': 14002,
 '一帶': 139,
 '傷心': 4756,
 '色入': 39345,
 '高樓': 51400,
 '有人': 23289,
 '樓上': 25180,
 '玉階空': 31382,
 '佇立': 3549,
 '宿鳥': 13846,
 '飛急': 50568,
 '何處': 3797,
 '歸程': 26005,
 '連短亭': 44987,
 '一作': 34,
 '短亭': 33975,
 '青天': 49338,
 '幾時': 16293,
 '我今': 19165,
 '停杯': 4540,
 '一問': 85,
 '人攀': 2776,
 '明月': 21861,
 '不可': 1095,
 '月行': 23274,
 '飛鏡': 50629,
 '臨丹': 38490,
 '綠煙': 36718,
 '海上': 27845,
 '寧知曉向': 14201,
 '雲間': 49106,
 '白兔': 32719,
 '搗藥秋': 20504,
 '嫦娥': 12848,
 '孤棲': 13044,
 '今人': 2906,
 '不見': 1437,
 '古時': 8146,
 '今月': 2965,
 '經照': 36650,
 '古人': 8117,
 '流水': 27645,
 '共看': 5627,
 '如此': 12473,
 '唯願': 9506,
 '當歌': 32285,
 '酒時': 46243,
 '月光': 23255,
 '照金樽裡': 30274,
 '燕草': 30406,
 '如碧絲': 12509,
 '秦桑低': 34736,
 '綠枝': 36692,
 '當君': 32234,
 '懷歸日': 18941,
 '斷腸時': 21402,
 '春風': 22255,
 '相識': 33499,
 '何事': 3603,
 '入羅': 5191,
 '家玉笛': 13765,
 '暗飛聲': 22808,
 '入春': 5164,
 '風滿': 50296,
 '洛城': 27541,
 '夜曲': 11466,
 '中聞折': 1834,
 '何人': 3604,
 '不起': 1475,
 '故園

In [14]:
reverse_voca = {v:k for k, v in vec.vocabulary_.items()}
reverse_voca
# 每個號碼對應的詞

{16053: '平林',
 29006: '漠漠',
 30177: '煙如織',
 14002: '寒山',
 139: '一帶',
 4756: '傷心',
 39345: '色入',
 51400: '高樓',
 23289: '有人',
 25180: '樓上',
 31382: '玉階空',
 3549: '佇立',
 13846: '宿鳥',
 50568: '飛急',
 3797: '何處',
 26005: '歸程',
 44987: '連短亭',
 34: '一作',
 33975: '短亭',
 49338: '青天',
 16293: '幾時',
 19165: '我今',
 4540: '停杯',
 85: '一問',
 2776: '人攀',
 21861: '明月',
 1095: '不可',
 23274: '月行',
 50629: '飛鏡',
 38490: '臨丹',
 36718: '綠煙',
 27845: '海上',
 14201: '寧知曉向',
 49106: '雲間',
 32719: '白兔',
 20504: '搗藥秋',
 12848: '嫦娥',
 13044: '孤棲',
 2906: '今人',
 1437: '不見',
 8146: '古時',
 2965: '今月',
 36650: '經照',
 8117: '古人',
 27645: '流水',
 5627: '共看',
 12473: '如此',
 9506: '唯願',
 32285: '當歌',
 46243: '酒時',
 23255: '月光',
 30274: '照金樽裡',
 30406: '燕草',
 12509: '如碧絲',
 34736: '秦桑低',
 36692: '綠枝',
 32234: '當君',
 18941: '懷歸日',
 21402: '斷腸時',
 22255: '春風',
 33499: '相識',
 3603: '何事',
 5191: '入羅',
 13765: '家玉笛',
 22808: '暗飛聲',
 5164: '入春',
 50296: '風滿',
 27541: '洛城',
 11466: '夜曲',
 1834: '中聞折',
 3604: '何人',
 1475: '不起',
 208

In [15]:
x_train
# 2731首詩，52294個詞，85677個非0位置

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [16]:
print(x_train)
# 為一sparse matrix(稀疏矩陣) -> 因矩陣太大(2731x52294)，因此只儲存非0矩陣

  (0, 16053)	1
  (0, 29006)	1
  (0, 30177)	1
  (0, 14002)	1
  (0, 139)	1
  (0, 4756)	1
  (0, 39345)	1
  (0, 51400)	1
  (0, 23289)	1
  (0, 25180)	1
  (0, 31382)	1
  (0, 3549)	1
  (0, 13846)	1
  (0, 50568)	1
  (0, 3797)	1
  (0, 26005)	1
  (0, 44987)	2
  (0, 34)	1
  (0, 33975)	1
  (1, 49338)	1
  (1, 16293)	1
  (1, 19165)	1
  (1, 4540)	1
  (1, 85)	1
  (1, 2776)	1
  :	:
  (2729, 8800)	1
  (2729, 6485)	1
  (2729, 3060)	1
  (2730, 25782)	1
  (2730, 6026)	1
  (2730, 36958)	1
  (2730, 15905)	1
  (2730, 44854)	1
  (2730, 29458)	1
  (2730, 2175)	1
  (2730, 31125)	1
  (2730, 40974)	1
  (2730, 35446)	1
  (2730, 49196)	1
  (2730, 6770)	1
  (2730, 48596)	1
  (2730, 7813)	1
  (2730, 1714)	1
  (2730, 39190)	1
  (2730, 28820)	1
  (2730, 38489)	1
  (2730, 28423)	1
  (2730, 51689)	1
  (2730, 8048)	1
  (2730, 23226)	1


#### <font color=red size=3>步驟四.5: 用vec.vocabulary_["，"]檢查標點符號(，)有沒有去掉 -> 執行出現錯誤即已去掉</font>

## 單純貝氏種類:
- Multinomial Naive Bayes(最常用): 李白骰骰子問題(整數特徵，固定幾個類別)
- Gaussian Naive Bayes: 把數值用常態分配擬合；使用時機: 特徵為連續值時 (ex.iris，但欄位不多，不必要也不常用)
- Complement Naive Bayes: 使用時機: 樣本不均衡時
- Bernoulli Naive Bayes: 欄位為二值型，似二值版本的Multinomial Naive Bayes(EX:是否 0，1)

#### <font color=red size=3>步驟五: 使用MultinomialNB訓練資料</font>

In [16]:
# 做公式出來
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### <font color=red size=3>步驟六: 使用accuracy_score檢測模型正確率</font>

In [17]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

0.8

#### <font color=red size=3>步驟七: 製作預測錯的列表 -> 混淆矩陣 confusion_matrix</font>
- confusion_matrix(y_test, pre) 參數不能代相反

In [18]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)

ori = ["李白", "杜甫", "白居易"]
#.format(): 字串格式化
r = ["{}(正確)".format(s) for s in ori]
c = ["{}(預測)".format(s) for s in ori]
pd.DataFrame(mat, 
             columns=c,
             index=r)

,李白(預測),杜甫(預測),白居易(預測)
李白(正確),8,1,1
杜甫(正確),1,8,1
白居易(正確),1,1,8


#### <font color=red size=3>步驟八: 製作詩詞分類預測器</font>

In [37]:
s = input("請輸入一首詩: ")
# 分詞, 記得要放入[]中
# 詞計數
print([poemcut(s)])
s = vec.transform([poemcut(s)])
print(s)

pre = clf.predict(s)[0]  # 拿出第一個
ans = reverse_trans[pre]
print("應該是: ", ans, "寫的")

請輸入一首詩: 黃河之水天上來
['黃 河之水 天上 來']
  (0, 11706)	1
  (0, 27199)	1
應該是:  李白 寫的
